# Drone Gym Environment and DQN Demo
## DM996 Group C, University of Strathclyde, 25/03/2021

This environment demonstrates A* algorithm in a randomly generated city environment (or a custom one - example included). The drone has a destination and must avoid obstacles such as buildings or other drones.

In [1]:
# general libraries
import numpy as np
import math
import random
import time

# box2D import libraries
import Box2D
from Box2D.b2 import fixtureDef
from Box2D.b2 import polygonShape
from Box2D.b2 import circleShape
from Box2D.b2 import contactListener

# OpenAI Gym import
import gym
from gym import error, spaces, utils
from gym.utils import seeding

# pyglet used for interacting with viewer window
import pyglet

# gym wrapper for creating environment visualiser
from gym.envs.classic_control import rendering

from gym import spaces

import matplotlib.pyplot as plt

import pandas as pd

In [27]:
# size of observable area from drone
OBSRV_WIDTH = 9
OBSRV_HEIGHT = 9

# define default shape for a drone
DRONE_SHAPE = [(0,0),(5,0),(5,5),(0,5)]

REPOSITION_ROAD_WINDOW_SIZE = 11 # FOR INITIAL DRONE/DESTINATION RE-POSITIONING - drone needs a REPOSITION_ROAD_WINDOW_SIZExREPOSITION_ROAD_WINDOW_SIZE window of clearance from any buildings

# define frames per second (fps) values
FPS = 1

# viewer window size
FRAME_WIDTH = 640
FRAME_HEIGHT = 480

class ContactDetector(contactListener):
    def __init__(self, env):
        contactListener.__init__(self)
        self.env = env

    def BeginContact(self, contact):
        if self.env.drones[0].dynamic_body == contact.fixtureA.body or self.env.drones[0].dynamic_body == contact.fixtureB.body:
            self.env.drone_contact = True


class Drone():
    def __init__(self):
        # drone initial position as 2D coordinates
        self.drone_position = None
        # current drone position as 2D coordinates
        self.drone_position_init = None
        # observations (2D image) for drone
        self.drone_observation = None
        # distance observation (float) of drone from destination
        self.drone_distance_observation = None
        
        
        # drone object in box2d
        self.dynamic_body = None
        
    def drone_step(self, action):
        # in this function, we update the position of the drone in box2D
        
        # apply a force to drone to update the position
        horizontal_force = action[0]
        vertical_force = action[1]
        self.dynamic_body.ApplyForceToCenter((horizontal_force,vertical_force),True)
        
    def update_position(self):
        # get position for this drone and save
        self.drone_position = self.dynamic_body.position
        
        self.drone_position[0] = round(self.drone_position[0])
        self.drone_position[1] = round(self.drone_position[1])
        
        self.dynamic_body_position = self.drone_position
        
class Node():
    def __init__(self,x=0,y=0):
        # Initialize the attributes of the node
        # Heuristics
        self.f = 0
        self.g = 0
        self.h = 0
        # Position
        self.position = (x,y)
        
    def __eq__(self, other):
        return self.position == other.position
    # May need a function that gets the node position in x and y
    
    # Would it be worth having a function that calculates the heuristics in this class?
class DroneEnv(gym.Env):
    #metadata = {'render.modes': ['human']}
    def __init__(self):
        ### PARAMETERS ###
        # random city generator parameters
        self.ROAD_WIDTH_INIT = 15
        self.ROAD_WIDTH_TOLERANCE = 3
        self.BUILDING_WIDTH_INIT = 50
        self.BUILDING_WIDTH_TOLERANCE = 7
        self.BUILDING_HEIGHT_INIT = 50
        self.BUILDING_HEIGHT_TOLERANCE = 7
                
        # drone parameters
        self.NO_DRONES = 5
        
        self.DRONE_START_POSITION = (10,10)
        self.DRONE_DESTINATION = (400,300)

        ### ENVIRONMENT OBJECTS ###
        # list of building vertices
        self.buildings = []
        self.parks = []
        
        self.STRATHY = 0
        
        # list of destination coordinates
        self.destination_box = []
        
        # list of drone objects
        self.drones = []
                
        self.done = False
        self.drone_contact = False
        self.reward = 0.0
        self.prev_reward = 0.0
        self.step_reward = 0.0
        
        ### BOX2D OBJECTS ###
        # box2d world
        self.world = Box2D.b2World((0, 0))
        # static bodies (buildings) in box2d world
        self.static_bodies = []
        
        self.world.contactListener = ContactDetector(self)
        
        ### VIEWER OBJECTS ###
        # define "Viewer" object and set width and height of window
        self.viewer = None
        self.viewer = rendering.Viewer(FRAME_WIDTH, FRAME_HEIGHT)
        self.drone_shape_viewer = None
        
        # Action and observation spaces
        self.action_space = spaces.Box(
            np.array([-1.0, -1.0]), np.array([+1.0, +1.0]), dtype=np.float32
        )  # horiz force, vertical force
        
        self.observation_space = spaces.Box(
            low=0, high=255, shape=(OBSRV_HEIGHT*OBSRV_WIDTH+1, 1), dtype=np.uint8
        )
        
    def initialise_environment(self):
        
        ### INITILIASE ENVIRONMENT ###
        self.create_buildings()
        #self.create_destination()
        self.render_init_city()
        self.create_destination()
        self.create_drones()
        self.render_init_drones()
        self.create_box2d_world()  
        
    def create_buildings(self):

        if(self.STRATHY == 1):
            # STRATHCLYDE BUILDING DEFINITIONS FOR 'FINAL DEMO'
            buildings_csv = pd.read_csv("Building_Data_CSV.csv")

            building = []

            building_names = buildings_csv['Building']

            x_coordinates = buildings_csv['X']
            y_coordinates = buildings_csv['Y']

            current_building = building_names[0]

            for i in range(len(x_coordinates)):
                vertex_index = 1
                if(building_names[i] == current_building):
                    building.append((float(x_coordinates[i]),float(y_coordinates[i])))
                else:
                    if(current_building == 'College Gardens' or current_building == 'Gardens'):
                        self.parks.append(building)
                    else:
                        self.buildings.append(building)

                    building = []
                    current_building = building_names[i]
                    building.append((float(x_coordinates[i]),float(y_coordinates[i])))
        else:             
            # RANDOM CITY GENERATION FOR TRAINING
            right_bound = 0
            top_bound = 0

            while(top_bound < FRAME_HEIGHT):
                top_bound_tracker = top_bound
                while(right_bound < FRAME_WIDTH):
                    road_width = self.ROAD_WIDTH_INIT + int(random.uniform(-self.ROAD_WIDTH_TOLERANCE,self.ROAD_WIDTH_TOLERANCE))
                    building_width = self.BUILDING_WIDTH_INIT + int(random.uniform(-30,30)) + int(random.uniform(-self.BUILDING_WIDTH_TOLERANCE,self.BUILDING_WIDTH_TOLERANCE))
                    building_height = self.BUILDING_HEIGHT_INIT + int(random.uniform(-30,30)) + int(random.uniform(-self.BUILDING_HEIGHT_TOLERANCE,self.BUILDING_HEIGHT_TOLERANCE))

                    bottom_left_coord = (right_bound+road_width,
                                         top_bound+road_width)

                    bottom_right_coord = (bottom_left_coord[0]+building_width,
                                          bottom_left_coord[1])

                    top_right_coord = (bottom_right_coord[0],
                                       bottom_right_coord[1]+building_height)

                    top_left_coord = (bottom_left_coord[0],
                                      top_right_coord[1])

                    building = [bottom_left_coord,bottom_right_coord,top_right_coord,top_left_coord]

                    # small chance of making the right bound smaller and hence potentially fusing the buildings or making a small alley the drone cant get down
                    if(random.uniform(0,1) > 0.1):
                        right_bound = bottom_right_coord[0]
                    else:
                        right_bound = bottom_right_coord[0] - road_width

                    if(right_bound < FRAME_WIDTH and top_right_coord[1] < FRAME_HEIGHT):
                        # small change to create a park
                        if(random.uniform(0,1) > 0.1):
                            self.buildings.append(building)
                        else:
                            self.parks.append(building)

                    # make sure we always have largest top bound to avoid buildings "fusing" together - with a small chance of not happening i.e. some fusing
                    if(top_left_coord[1] > top_bound_tracker and random.uniform(0,1) > 0.2):
                        top_bound_tracker = top_left_coord[1]

                # small chance of fusing the buildings vertically (or making them very close together)
                if(random.uniform(0,1) > 0.1):
                    top_bound = top_bound_tracker
                else:
                    top_bound = top_bound_tracker - self.ROAD_WIDTH_INIT

                right_bound = 0
            
    def create_destination(self):
        # get new position of destination on a road
        self.DRONE_DESTINATION = self.re_position_coord_on_road(self.DRONE_DESTINATION)

        drone_dest = ((self.DRONE_DESTINATION[0]-3,self.DRONE_DESTINATION[1]-3),
                    (self.DRONE_DESTINATION[0]+4,self.DRONE_DESTINATION[1]-3),
                    (self.DRONE_DESTINATION[0]+4,self.DRONE_DESTINATION[1]+4),
                    (self.DRONE_DESTINATION[0]-3,self.DRONE_DESTINATION[1]+4))
    
        self.destination_box.append(drone_dest)
        
        # place destination into viewer window
        self.viewer.add_geom(self.viewer.draw_polygon(self.destination_box[0],1,color=[0.5,0.2,0.9])) 
         
    def re_position_coord_on_road(self,START_POSITION_):
        # this function takes a starting coordinate for the drone, determines whether it is merged with a building
        # (or other drone) and returns the nearest coordinate where it is not colliding
        START_POSITION = (0,0)
        
        START_POSITION = (START_POSITION_[1],START_POSITION_[0])
        
        # get image of environment
        self.viewer.render()
        image_data_buffer = pyglet.image.get_buffer_manager().get_color_buffer().get_image_data().get_data()         
        image_data = np.frombuffer(image_data_buffer,dtype=np.uint8,count=-1,offset=0) # pixel data is in one big line of values
        pixel_data = image_data.reshape(len(image_data)//4,4) # get individual pixel data and store in elements in an array
        pixel_data = pixel_data.reshape(FRAME_HEIGHT,FRAME_WIDTH,4)
        
        city_image = pixel_data
        
        window_row_start = START_POSITION[0]-math.floor(REPOSITION_ROAD_WINDOW_SIZE/2)+2
        window_row_end = window_row_start+REPOSITION_ROAD_WINDOW_SIZE

        if(window_row_start < 0):
            window_row_start = 0
        elif(window_row_end > FRAME_HEIGHT):
            window_row_end = FRAME_HEIGHT  

        window_col_start = START_POSITION[1]-math.floor(REPOSITION_ROAD_WINDOW_SIZE/2)+2
        window_col_end = window_col_start+REPOSITION_ROAD_WINDOW_SIZE

        if(window_col_start < 0):
            window_col_start = 0
        elif(window_col_end > FRAME_WIDTH):
            window_col_end = FRAME_WIDTH

        city_image_section = city_image[window_row_start:window_row_end,window_col_start:window_col_end,:]

        reposition_indicator = 0

        # loop over rows
        for i in range(np.shape(city_image_section)[0]):
            # loop over columns
            for j in range(np.shape(city_image_section)[1]):
                if(city_image_section[i,j,0] != 166):
                    # i.e. a building - not the ground
                    reposition_indicator = 1
                    break
            if(reposition_indicator == 1):
                break
        
        if(reposition_indicator == 0):
            #print("test1")
            return START_POSITION_
        else:
            # perform an averaging convolution/correlation to find areas where the drone can spawn (outwith a building)
            # if the convoloved result has a pixel equal to the average (mean) of the ground pixels then we know there is a window at that point the drone can spawn at
            conv_row_start = math.floor(REPOSITION_ROAD_WINDOW_SIZE/2)
            conv_row_end = FRAME_HEIGHT - conv_row_start

            conv_col_start = math.floor(REPOSITION_ROAD_WINDOW_SIZE/2)
            conv_col_end = FRAME_WIDTH - conv_col_start

            convolved_result = np.zeros(shape=(FRAME_HEIGHT,FRAME_WIDTH))
            avg_factor = REPOSITION_ROAD_WINDOW_SIZE**2

            stride = 5

            # implementation of mean filter with kernel of size DRONE_WINDOW_SIZExDRONE_WINDOW_SIZE
            for i in range(FRAME_HEIGHT):
                for j in range(FRAME_WIDTH):
                    if(i >= conv_row_start and i < conv_row_end and j >= conv_col_start and j < conv_col_end):
                        if(i % stride == 0 and j % stride == 0):
                            window_row_start_idx = i - math.floor(REPOSITION_ROAD_WINDOW_SIZE/2)
                            window_col_start_idx = j - math.floor(REPOSITION_ROAD_WINDOW_SIZE/2)
                            # loop over kernel rows
                            for k1 in range(REPOSITION_ROAD_WINDOW_SIZE):
                                # loop over kernel columns
                                for k2 in range(REPOSITION_ROAD_WINDOW_SIZE):
                                    convolved_result[i,j] += city_image[window_row_start_idx+k1,window_col_start_idx+k2,0]

                            convolved_result[i,j] = convolved_result[i,j] / avg_factor

            start_coords = []
            # new coordinate will stay the same if an appriopriate position cannot be found
            new_coord = START_POSITION

            # find coordinates where drone is allowed to start
            for i in range(FRAME_HEIGHT):
                for j in range(FRAME_WIDTH):
                    if(convolved_result[i,j] == 166):
                        start_coords.append((i,j))

            # find closest index to originally requrested position
            if(len(start_coords) > 0):
                smallest_distance = math.sqrt((START_POSITION[0]-start_coords[0][0])**2 + (START_POSITION[1]-start_coords[0][1])**2)
            for i in range(len(start_coords)):
                if(i > 0):
                    distance = math.sqrt((START_POSITION[0]-start_coords[i][0])**2 + (START_POSITION[1]-start_coords[i][1])**2)
                    if(distance < smallest_distance):
                        smallest_distance = distance
                        new_coord = start_coords[i]
            
            new_coord_return = (new_coord[1],new_coord[0])
            return new_coord_return
            
    def create_drones(self):
        ### create the first drone
        # create instance of Drone class
        drone = Drone()
        
        # first drone position set manually - check if the drone is starting in a building and if so, move it closeby out of building
        drone.drone_position_init = self.re_position_coord_on_road(self.DRONE_START_POSITION)
        drone.drone_position = drone.drone_position_init
        drone.drone_observation = np.zeros(shape=(OBSRV_WIDTH,OBSRV_HEIGHT,4),dtype=np.uint8)
        # add drone to list
        self.drones.append(drone)
        
        # loop over remaining drones
        for i in range(self.NO_DRONES):
            if(i > 0):
                # create instance of Drone class
                drone = Drone()
                # define drone initial positions and observation windows of drones (square around)
                drone.drone_position_init = (random.uniform(0, FRAME_WIDTH-1),random.uniform(0, FRAME_HEIGHT-1))
                drone.drone_position = drone.drone_position_init
                drone.drone_observation = np.zeros(shape=(OBSRV_WIDTH,OBSRV_HEIGHT,4),dtype=np.uint8)
                # add drone to list
                self.drones.append(drone)
    
    def create_box2d_world(self):
        # create static bodies (buildings) in world
        for i in range(len(self.buildings)):
            self.static_bodies.append(self.world.CreateStaticBody(fixtures=fixtureDef(shape=polygonShape(vertices=self.buildings[i]))))
    
        # create dynamic bodies (drones) in world
        for i in range(len(self.drones)):
            self.drones[i].dynamic_body = self.world.CreateDynamicBody(
            position=self.drones[i].drone_position_init, # MAY HAVE TO CHANGE THIS INITIAL POSITION FROM PIXELS TO METRES
            angularDamping = 0.1,
            angle=0,
            fixtures=fixtureDef(shape=polygonShape(vertices=DRONE_SHAPE)))
    
   # Implement the astar
    def astar(self):
        # Function that defines a basic astar pathfinder
       
        # Instantiate a Node to act as the starting node
        # specifying the co-ordinate as the current co-ordinates of the drone
        start_node = Node(self.drones[0].drone_position[0],self.drones[0].drone_position[1])
        
        # Instantiate another Node to act as the end node
        # this time using the coordinates of the DRONE_DESTINATION
        end_node = Node(self.DRONE_DESTINATION[0],self.DRONE_DESTINATION[1])
        
        # Create open and closed lists
        # These are used to evaluate the lowest cost path
        open_list = []
        closed_list = []
        
        # Add the drone initial position to the open list
        open_list.append(start_node)
        
        # Show drone at start node
        self.render()
        
        # Loop through the open list to find the lowest cost node
        while len(open_list) > 0:
            
            # move all drones randomly
            drone_original_positions = []
            for i in range(len(self.drones)-1):
                pos = self.drones[i+1].dynamic_body.position
                drone_original_positions.append(pos)
                self.drones[i+1].dynamic_body.position = (pos[0] + random.randint(-2,2),pos[1] + random.randint(-2,2))
                self.drones[i+1].update_position()
            
            self.render()
            
            # Set first entry in open list to current node
            current_node = open_list[0]
            current_index = 0
            for index, item in enumerate(open_list):
                if item.f < current_node.f: # If node costs less
                    current_node = item # update open list
                    current_index = index
                    
            # Add the lowest cost node in the open list to the closed list
            open_list.pop(current_index)
            closed_list.append(current_node)
            
            # Check if the drone destination has been reached
            #if current_node.position == end_node.position:
            if((abs(current_node.position[0] - end_node.position[0]) < 4) and
               (abs(current_node.position[1] - end_node.position[1]) < 4)):
                
                # Show the drone at final position
                self.drones[0].dynamic_body.position = end_node.position
                self.drones[0].update_position()
                self.world.Step(1/FPS,1,1)
                self.render()
                
                break
               
            # Generate 'children' as the adjacent points of the environment maze
            # Manually update position in each direction
            children = []
            original_position = current_node.position
            
            search = 3
            
            for candidate_position in [(0, -search), (0, search), (-search, 0), (search, 0), (-search, -search), (-search, search), (search, -search), (search, search)]:
                
                # Save position (check current_node)
                new_position = (int(current_node.position[0]) + int(candidate_position[0]),int(current_node.position[1]) + int(candidate_position[1])) # Check
                self.drones[0].dynamic_body.position = new_position
            
                # Make sure within range of the environment
                if(new_position[0] < 0 or new_position[0] > FRAME_WIDTH or
                   new_position[1] < 0 or new_position[1] > FRAME_HEIGHT):
                    continue
            
                # Apply Box2D world step
                self.world.Step(1/FPS,1,1)

                # Check if there is contact
                if self.drone_contact is True:
                    # function to reset
                    # Put the drone back to where it was before a force was applied
                    self.drones[0].dynamic_body.position = original_position
                
                    # reset contact back to false
                    self.drone_contact = False
                
                    # Apply Box2D world step
                    self.world.Step(1/FPS,1,1)
                    
                    
                else:
                    # Having checked the position is valid, add to children
                    new_node = Node(new_position[0],new_position[1])
                    children.append(new_node)

                    # function to reset
                    # Put the drone back to where it was before a force was applied
                    self.drones[0].dynamic_body.position = original_position

                    # reset contact back to false
                    self.drone_contact = False

                    # Apply Box2D world step
                    self.world.Step(1/FPS,1,1)
                
             #Evaluate each of the children
            for child in children:
                
                # Check if the child is on the closed list
                for closed_child in closed_list:
                    if child == closed_child:
                        continue
                        
                # Calculate the f, g, and h values for the children
                child.g = current_node.g + 1 # distance increased by 1
                child.h = math.sqrt((child.position[0]-end_node.position[0])**2 + (child.position[1]-end_node.position[1])**2) # Revised using math
                child.f = child.g + child.h      # total cost of the node
                
                # Child is already in the open list
                for open_node in open_list:
                    if child == open_node and child.g > open_node.g: # check if more advanced node
                        continue
                        
                # Add the child to the open list
                open_list.append(child)
                
                # Show the environment updated at t+1
                self.drones[0].dynamic_body.position = child.position
                self.drones[0].update_position()
                self.world.Step(1/FPS,1,1)
                
                if child.h <= search:
                    self.render()
                    break
                else:
                    continue
                
           # break
                        
    def reset(self):
        self.drone_contact = False
        self.done = False
        

        for i in range(len(self.drones)):
            self.world.DestroyBody(self.drones[i].dynamic_body)
            
            self.drones[i].dynamic_body = self.world.CreateDynamicBody(
            position=self.drones[i].drone_position_init, # MAY HAVE TO CHANGE THIS INITIAL POSITION FROM PIXELS TO METRES
            angularDamping = 0.1,
            angle=0,
            fixtures=fixtureDef(shape=polygonShape(vertices=DRONE_SHAPE)))
    
        # set state as observation window (per racecar env)
        state = (self.drones[0].drone_observation, self.drones[0].drone_distance_observation)
        state = self.process_states(state)
        
        return state
        
        
    def render_init_city(self):
        # draw background
        self.viewer.add_geom(self.viewer.draw_polygon([(0,0),(FRAME_WIDTH,0),(FRAME_WIDTH,FRAME_HEIGHT),(0,FRAME_HEIGHT)],1,color=[0.65,0.65,0.65]))
        
        # place buildings into viewer window
        for i in range(len(self.buildings)):
            self.viewer.add_geom(self.viewer.draw_polygon(self.buildings[i],1,color=[0.85,0.85,0.85]))   
            
        # place parks into viewer
        for i in range(len(self.parks)):
            self.viewer.add_geom(self.viewer.draw_polygon(self.parks[i],1,color=[0.7,0.85,0.15]))           
    
        # update viewer window and wait before adding drones
        self.viewer.render()
        time.sleep(1)
        
    def render_init_drones(self):

        # place drone into viewer window
        drone_positions = []
        for i in range(len(self.drones)):
            drone_positions.append(self.drones[i].drone_position_init)

        # needs to consider the shape of the drone plus the current offset
        self.drone_shape_viewer = []
        for j in range(len(self.drones)): 
            self.drone_shape_viewer.append(DRONE_SHAPE.copy())

        # draw initial positions of drones
        for j in range(len(self.drones)):
            for k in range(len(DRONE_SHAPE)):
                self.drone_shape_viewer[j][k] = (DRONE_SHAPE[k][0] + drone_positions[j][0],DRONE_SHAPE[k][1] + drone_positions[j][1])    

        # add drones to viewer
        
        # draw drone
        drone_centre = (self.drones[0].drone_position[0]+2,self.drones[0].drone_position[1]+2)
        self.draw_drone(drone_centre,(255,0,0))
        
        # remaining drones - same colour
        for j in range(len(self.drones)-1):          
            drone_centre = (self.drones[j+1].drone_position[0]+2,self.drones[j+1].drone_position[1]+2)
            self.draw_drone(drone_centre,(0,0,255))
    
        # render viewer window and wait a second
        self.viewer.render()
    
        time.sleep(1)        
    
    def render(self):
        # in this loop, we update the position of the drones in the viewer window
        for i in range(len(self.drones)):
            # inner loop for individual drone geometries
            for j in range(5):
                del self.viewer.geoms[-1]
        
        for j in range(len(self.drones)):
            for k in range(len(DRONE_SHAPE)):
                self.drone_shape_viewer[j][k] = (DRONE_SHAPE[k][0] + self.drones[j].drone_position[0],DRONE_SHAPE[k][1] + self.drones[j].drone_position[1])
                        
            # add drone to viewer
            if(j == 0):       
                # draw drone
                drone_centre = (self.drones[j].drone_position[0]+2,self.drones[j].drone_position[1]+2)
                self.draw_drone(drone_centre,(255,0,0))
            else:           
                drone_centre = (self.drones[j].drone_position[0]+2,self.drones[j].drone_position[1]+2)
                self.draw_drone(drone_centre,(0,0,255))
            
        # render viewer window
        self.viewer.render()
        
        
    def draw_drone(self,drone_centre,drone_colour): 
        # main chassis
        chassis_bottom_left_coord = (drone_centre[0]-1,drone_centre[1]-3)
        chassis_bottom_right_coord = (drone_centre[0]+2,drone_centre[1]-3)
        chassis_top_right_coord = (drone_centre[0]+2,drone_centre[1]+4)
        chassis_top_left_coord = (drone_centre[0]-1,drone_centre[1]+4)

        self.viewer.add_geom(self.viewer.draw_polygon((chassis_bottom_left_coord,
                                                       chassis_bottom_right_coord,
                                                       chassis_top_right_coord,
                                                       chassis_top_left_coord,
                                                       chassis_bottom_left_coord),1,color=[drone_colour[0],drone_colour[1],drone_colour[2]]))
        
        # bottom left propeller
        circle_geom = rendering.make_circle(radius=3, filled=False)
        circle_geom.set_color(0,0,0)        
        for k in range(len(circle_geom.v)):
            circle_geom.v[k] = (circle_geom.v[k][0] + drone_centre[0]-4,circle_geom.v[k][1] + drone_centre[1]-4)
        self.viewer.add_geom(circle_geom)
                
        # bottom right propeller
        circle_geom = rendering.make_circle(radius=3, filled=False)
        circle_geom.set_color(0,0,0)        
        for k in range(len(circle_geom.v)):
            circle_geom.v[k] = (circle_geom.v[k][0] + drone_centre[0]+5,circle_geom.v[k][1] + drone_centre[1]-4)
        self.viewer.add_geom(circle_geom)
                
        # top left propeller
        circle_geom = rendering.make_circle(radius=3, filled=False)
        circle_geom.set_color(0,0,0)        
        for k in range(len(circle_geom.v)):
            circle_geom.v[k] = (circle_geom.v[k][0] + drone_centre[0]-4,circle_geom.v[k][1] + drone_centre[1]+5)
        self.viewer.add_geom(circle_geom)
                
        # top right propeller
        circle_geom = rendering.make_circle(radius=3, filled=False)
        circle_geom.set_color(0,0,0)        
        for k in range(len(circle_geom.v)):
            circle_geom.v[k] = (circle_geom.v[k][0] + drone_centre[0]+5,circle_geom.v[k][1] + drone_centre[1]+5)
        self.viewer.add_geom(circle_geom)
        
    def close(self):
        self.viewer.close()

## Demonstrate A* algorithm operating in the environment

In [36]:
# instantiate environment
drone_env =  DroneEnv()

# random city generator parameters - need to be selected carefully so buildings maintain normal shape compatible with box2d
drone_env.ROAD_WIDTH_INIT = 18
drone_env.ROAD_WIDTH_TOLERANCE = 3
drone_env.BUILDING_WIDTH_INIT = 50
drone_env.BUILDING_WIDTH_TOLERANCE = 7
drone_env.BUILDING_HEIGHT_INIT = 50
drone_env.BUILDING_HEIGHT_TOLERANCE = 7

# set number of drones in environment (includes one we are controlling)
drone_env.NO_DRONES = 10

# set start and end positions of drone we are controlling
#drone_env.DRONE_START_POSITION = (350,200)
#drone_env.DRONE_END_POSITION = (400,300)
drone_env.DRONE_START_POSITION = (int(random.uniform(20,FRAME_WIDTH-21)),int(random.uniform(20,FRAME_HEIGHT-21)))
drone_env.DRONE_DESTINATION = (int(random.uniform(20,FRAME_WIDTH-21)),int(random.uniform(20,FRAME_HEIGHT-21)))

# needs to be called after the parameters have been set
drone_env.initialise_environment()

#astar
drone_env.astar()

drone_env.close()

KeyboardInterrupt: 

In [37]:
drone_env.close()